Programación Defensiva

## Questions

- ¿Cómo puedo hacer mis programas más fiables?

## Objetivos

- Explicar lo que es una afirmación.

- Agregar aserciones que comprueben que el estado del programa es correcto.

- Agregue correctamente las afirmaciones de precondición y postcondición a las funciones.

- Explicar qué es el desarrollo basado en pruebas y utilizarlo al crear nuevas funciones.


Nuestras lecciones anteriores han introducido las herramientas básicas de programación: variables y listas, archivo E/S, bucles, condicionales y funciones. Lo que no han hecho es mostrarnos cómo saber si un programa está recibiendo la respuesta correcta, y cómo saber si todavía está recibiendo la respuesta correcta a medida que hacemos cambios en ella.

Para lograrlo, debemos:

- Escribir programas que comprueben su propio funcionamiento.
- Escribir y ejecutar pruebas para funciones ampliamente utilizadas.
- Asegúrese de que sabemos lo que significa "correcto".

La buena noticia es que hacer estas cosas acelerará nuestra programación, no la ralentizará. Como en la carpintería real - el tipo hecho con madera - el tiempo ahorrado midiendo cuidadosamente antes de cortar un pedazo de madera es mucho mayor que el tiempo que toma la medición Como dice el refrán, "mida dos veces, corte una".

# Aserciones

El primer paso para obtener las respuestas correctas de nuestros programas es asumir que los errores ocurrirán y protegerse contra ellos. Esto se llama programación defensiva, y la forma más común de hacerlo es agregar aserciones a nuestro código para que se verifique a medida que se ejecuta. Una afirmación es simplemente una declaración de que algo debe ser cierto en un cierto punto de un programa. Cuando Python ve uno, evalúa la condición de la afirmación. Si es cierto, Python no hace nada, pero si es falso, Python detiene el programa inmediatamente e imprime el mensaje de error si se proporciona uno. Por ejemplo, esta pieza de código se detiene tan pronto como el bucle encuentra un valor que no es positivo:


In [1]:
numbers = [1.5, 2.3, 0.7, -0.001, 4.4]
total = 0.0
for num in numbers:
    assert num > 0.0, 'Data should only contain positive values'
    total += num
print('total is:', total)

AssertionError: Data should only contain positive values

Programas como el navegador Firefox están llenos de afirmaciones: 10-20% del código que contienen están ahí para comprobar que el otro 80-90% están funcionando correctamente. En términos generales, las afirmaciones se dividen en tres categorías:

1. Una condición previa es algo que debe ser cierto al comienzo de una función para que funcione correctamente.

2. Una postcondición es algo que la función garantiza que es cierto cuando termina.

3. Un invariante es algo que siempre es cierto en un punto particular dentro de una pieza de código.

Por ejemplo, supongamos que estamos representando rectángulos usando una tupla de cuatro coordenadas `(x0, y0, x1, y1)`, representando las esquinas inferior izquierda y superior derecha del rectángulo. Para hacer algunos cálculos, necesitamos normalizar el rectángulo para que la esquina inferior izquierda esté en el origen y el lado más largo tenga 1.0 unidades de largo. Esta función hace eso, pero comprueba que su entrada está correctamente formateada y que su resultado tiene sentido:

In [1]:
def normalize_rectangle(rect):
    """Normalizes a rectangle so that it is at the origin and 1.0 units long on its longest axis.
    Input should be of the format (x0, y0, x1, y1).
    (x0, y0) and (x1, y1) define the lower left and upper right corners
    of the rectangle, respectively."""
    assert len(rect) == 4, 'Rectangles must contain 4 coordinates'
    x0, y0, x1, y1 = rect
    assert x0 < x1, 'Invalid X coordinates'
    assert y0 < y1, 'Invalid Y coordinates'

    dx = x1 - x0
    dy = y1 - y0
    if dx > dy:
        scaled = float(dx) / dy
        upper_x, upper_y = 1.0, scaled
    else:
        scaled = float(dx) / dy
        upper_x, upper_y = scaled, 1.0

    assert 0 < upper_x <= 1.0, 'Calculated upper X coordinate invalid'
    assert 0 < upper_y <= 1.0, 'Calculated upper Y coordinate invalid'

    return (0, 0, upper_x, upper_y)

Las condiciones previas en las líneas 6, 8 y 9 capturan entradas no válidas:

In [2]:
print(normalize_rectangle( (0.0, 1.0, 2.0) )) # missing the fourth coordinate

AssertionError: Rectangles must contain 4 coordinates

In [3]:
print(normalize_rectangle( (4.0, 2.0, 1.0, 5.0) )) # X axis inverted

AssertionError: Invalid X coordinates

Las condiciones posteriores en las líneas 20 y 21 nos ayudan a detectar errores al decirnos cuándo nuestros cálculos podrían haber sido incorrectos. Por ejemplo, si normalizamos un rectángulo que es más alto que ancho todo parece estar bien:

In [4]:
print(normalize_rectangle( (0.0, 0.0, 1.0, 5.0) ))

(0, 0, 0.2, 1.0)


pero si normalizamos uno que es más ancho que alto, la afirmación se activa:

In [5]:
print(normalize_rectangle( (0.0, 0.0, 5.0, 1.0) ))

AssertionError: Calculated upper Y coordinate invalid

Releyendo nuestra función, nos damos cuenta de que la línea 14 debe dividir `dy` por `dx` en lugar de `dx` por `dy`. En un cuaderno Jupyter, puede mostrar los números de línea escribiendo <kbd>shift</kbd>+<kbd>L</kbd>. Si hubiéramos omitido la afirmación al final de la función, habríamos creado y devuelto algo que tenía la forma correcta como una respuesta válida, pero no lo era. Detectar y depurar que casi con seguridad habría tomado más tiempo a largo plazo que escribir la afirmación.

Pero las aserciones no son solo para detectar errores: también ayudan a la gente a entender los programas. Cada afirmación le da a la persona que lee el programa la oportunidad de comprobar (conscientemente o de otra manera) que su comprensión coincide con lo que el código está haciendo.

La mayoría de los buenos programadores siguen dos reglas cuando agregan aserciones a su código. La primera es fallar temprano, fallar a menudo. Cuanto mayor sea la distancia entre cuándo y dónde se produce un error y cuando se nota, más difícil será depurar el error, por lo que un buen código detecta errores lo antes posible.

La segunda regla es convertir los errores en aserciones o pruebas. Cada vez que corrige un error, escriba una afirmación que detecte el error si lo vuelve a cometer. Si cometiste un error en una pieza de código, las probabilidades son buenas de que hayas cometido otros errores cerca o cometerás el mismo error (o uno relacionado) la próxima vez que lo cambies. Escribir aserciones para comprobar que no has retrocedido (es decir, no has reintroducido un problema antiguo) puede ahorrar mucho tiempo a largo plazo, y ayuda a advertir a las personas que están leyendo el código (incluido tu yo futuro) que este bit es complicado.

# Desarrollo basado en pruebas

Una afirmación comprueba que algo es cierto en un punto particular del programa. El siguiente paso es verificar el comportamiento general de una pieza de código, es decir, asegurarse de que produce la salida correcta cuando se le da una entrada particular. Por ejemplo, supongamos que necesitamos encontrar dónde dos o más series temporales se superponen. El rango de cada serie temporal se representa como un par de números, que son el tiempo en que el intervalo comenzó y terminó. La salida es el rango más grande que todos ellos incluyen:

![ranges](media/python-overlapping-ranges.svg)

La mayoría de los programadores novatos resolverían este problema así:

1. Escribir una función range_overlap.
2. Llámelo interactivamente en dos o tres entradas diferentes.
3. Si produce la respuesta incorrecta, corrija la función y vuelva a ejecutar esa prueba.

Esto claramente funciona - después de todo, miles de científicos lo están haciendo ahora mismo - pero hay una mejor manera:

1. Escribir una función corta para cada prueba.
2. Escribir una función range_overlap que deba superar dichas pruebas.
3. Si range_overlap produce respuestas erróneas, arréglelo y vuelva a ejecutar las funciones de prueba.

Escribir las pruebas *antes* de escribir la función que ejercen se llama desarrollo basado en pruebas (TDD). Sus defensores creen que produce mejor código más rápido porque:

1. Si las personas escriben pruebas después de escribir lo que se va a probar, están sujetas a un sesgo de confirmación, es decir, escriben pruebas subconscientemente para demostrar que su código es correcto, en lugar de encontrar errores.
2. Escribir pruebas ayuda a los programadores a averiguar qué se supone que debe hacer la función.

Aquí hay tres funciones de prueba para range_overlap:

In [6]:
assert range_overlap([ (0.0, 1.0) ]) == (0.0, 1.0)
assert range_overlap([ (2.0, 3.0), (2.0, 4.0) ]) == (2.0, 3.0)
assert range_overlap([ (0.0, 1.0), (0.0, 2.0), (-1.0, 1.0) ]) == (0.0, 1.0)

NameError: name 'range_overlap' is not defined

Cuando ejecutamos esto, tenemos un error. El error es tranquilizador: no hemos escrito range_overlap todavía, así que si las pruebas pasan, sería una señal de que alguien más tenía y que estábamos utilizando accidentalmente su función.

Y como ventaja de escribir estas pruebas, hemos definido implícitamente cómo se ven nuestra entrada y salida: esperamos una lista de pares como entrada y producimos un solo par como salida.

Sin embargo, falta algo importante. No tenemos pruebas para el caso de que los rangos no se solapen en absoluto:

```Python
assert range_overlap([ (0.0, 1.0), (5.0, 6.0) ]) == ???
```

¿Qué debería hacer range_overlap en este caso: fallar con un mensaje de error, producir un valor especial como (0.0, 0.0) para indicar que no hay superposición, o algo más? Cualquier implementación real de la función hará una de estas cosas; escribir las pruebas primero nos ayuda a averiguar cuál es el mejor *antes* estamos emocionalmente involucrados en lo que sea que escribimos antes de darnos cuenta de que había un problema.

¿Y qué hay de este caso?

```Python
assert range_overlap([ (0.0, 1.0), (1.0, 2.0) ]) == ???
```

¿Dos segmentos que se tocan en sus puntos finales se superponen o no? Los matemáticos suelen decir "sí", pero los ingenieros suelen decir "no". La mejor respuesta es "lo que sea más útil en el resto de nuestro programa", pero de nuevo, cualquier implementación real de `range_overlap` va a hacer algo, y lo que sea debe ser consistente con lo que hace cuando no hay superposición en absoluto.

Dado que estamos planeando usar el rango esta función regresa como el eje X en un gráfico de series temporales, decidimos que:

1. cada solapamiento debe tener una anchura distinta de cero, y
2. devolveremos el valor especial `Ninguno` cuando no haya superposición.

`Ninguno` está integrado en Python y significa "nada aquí" (Otros lenguajes a menudo llaman al valor equivalente `null` o `nil`). Con esa decisión tomada, podemos terminar de escribir nuestras dos últimas pruebas:


In [7]:
assert range_overlap([ (0.0, 1.0), (5.0, 6.0) ]) == None
assert range_overlap([ (0.0, 1.0), (1.0, 2.0) ]) == None

NameError: name 'range_overlap' is not defined

De nuevo, tenemos un error porque no hemos escrito nuestra función, pero ahora estamos listos para hacerlo:

In [8]:
def range_overlap(ranges):
    """Return common overlap among a set of [left, right] ranges."""
    max_left = 0.0
    min_right = 1.0
    for (left, right) in ranges:
        max_left = max(max_left, left)
        min_right = min(min_right, right)
    return (max_left, min_right)

Tómese un momento para pensar por qué calculamos el punto final izquierdo del solapamiento como el máximo de los puntos finales izquierdos de entrada, y el punto final derecho de superposición como el mínimo de los puntos finales derechos de entrada. Ahora nos gustaría volver a realizar nuestras pruebas, pero están dispersas en tres celdas diferentes. Para hacerlas más fáciles, pongámoslas todas en una función:

In [9]:
def test_range_overlap():
    assert range_overlap([ (0.0, 1.0), (5.0, 6.0) ]) == None
    assert range_overlap([ (0.0, 1.0), (1.0, 2.0) ]) == None
    assert range_overlap([ (0.0, 1.0) ]) == (0.0, 1.0)
    assert range_overlap([ (2.0, 3.0), (2.0, 4.0) ]) == (2.0, 3.0)
    assert range_overlap([ (0.0, 1.0), (0.0, 2.0), (-1.0, 1.0) ]) == (0.0, 1.0)
    assert range_overlap([]) == None

Ahora podemos probar `range_overlap` con una sola llamada de función:

In [10]:
test_range_overlap()


AssertionError: 

La primera prueba que se suponía que iba a producir `Ninguno` falla, por lo que sabemos que algo está mal con nuestra función. No sabemos si las otras pruebas pasaron o fallaron porque Python detuvo el programa tan pronto como detectó el primer error. Aún así, alguna información es mejor que ninguna, y si rastreamos el comportamiento de la función con esa entrada, nos damos cuenta de que estamos inicializando `max_left` y `min_right` a 0.0 y 1.0 respectivamente, independientemente de los valores de entrada. Esto viola otra regla importante de programación: siempre inicializar a partir de datos.

Ejercicio 1* - Condiciones previas y posteriores

Supongamos que está escribiendo una función llamada `promedio` que calcula el promedio de los números en una lista. ¿Qué condiciones previas y condiciones posteriores escribirías para ello? 

In [1]:
def average(numbers):
    result = 0
    counter = 0
    for num in numbers:
        result += 0
        counter += 1
    return result/counter

# Ejercicio 2* - Pruebas de aserciones

Dada una secuencia de un número de coches, la función `get_total` devuelve el número total de coches.

```Python
get_total([1, 2, 3, 4])
```
```
>>> 10
```

---
```Python
get_total(['a', 'b', 'c'])
```
```
ValueError: invalid literal for int() with base 10: 'a'
```

Explique en palabras lo que las aserciones en esta función comprueban, y para cada una, dé un ejemplo de entrada que hará que esa aserción falle.


In [2]:
def get_total(values):
    assert len(values) > 0
    for element in values:
        assert int(element) > 0, "element must be > 0"
    total = sum(values)
    assert total > 0
    return total

get_total([7, 8])

15

# Puntos clave

- Programar defensivamente, es decir, asumir que van a surgir errores, y escribir código para detectarlos cuando lo hagan.

- Poner aserciones en los programas para comprobar su estado a medida que se ejecutan, y para ayudar a los lectores a entender cómo se supone que esos programas funcionan.

- Utilice precondiciones para comprobar que las entradas a una función son seguras de usar.

- Utilice postcondiciones para comprobar que la salida de una función es segura de usar.

- Escribir pruebas antes de escribir código para ayudar a determinar exactamente lo que ese código se supone que debe hacer.